In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys
import shutil

from hloc import (
    extract_features,
    match_features,
    match_dense,
    reconstruction,
    visualization,
    pairs_from_all,
    localize_sfm
)

ROOT = Path().absolute()
# add root to path in order to execute scripts
if str(ROOT) not in sys.path:
  sys.path.append(str(ROOT))

In [ ]:
images = Path("datasets/framlingham2/images/")

outputs = Path("outputs/sfm/")
sfm_pairs = outputs / "pairs.txt"
sfm_dir = outputs / "framlingham_s2d_nn"
sfm_images = sfm_dir / "images"

# retrieval_conf = extract_features.confs["netvlad"]
feature_conf = extract_features.confs["s2dnet"]
matcher_conf = match_features.confs["NN-ratio"]
# matcher_conf = match_dense.confs["loftr"]

In [ ]:
pairs_from_all.main(sfm_pairs, images)

In [ ]:
feature_path = extract_features.main(feature_conf, images, outputs)
#feature_path = Path(outputs, feature_conf["output"] + ".h5")

In [ ]:
match_path = match_features.main(
    conf=matcher_conf,
    pairs=sfm_pairs,
    features=feature_conf["output"],
    export_dir=outputs,
)

"""
feature_path, match_path = match_dense.main(
  conf=matcher_conf, 
  pairs=sfm_pairs, 
  image_dir=images, 
  export_dir=outputs,
  features_ref=feature_path
)
"""

## 3D reconstruction
Run COLMAP on the features and matches.

In [ ]:
model = reconstruction.main(sfm_dir, sfm_images, sfm_pairs, feature_path, match_path)

In [ ]:
'''
localize_sfm.main(
    sfm_dir,
    dataset / "queries/*_time_queries_with_intrinsics.txt",
    loc_pairs,
    features,
    loc_matches,
    results,
    covisibility_clustering=False,
)  # not required with SuperPoint+SuperGlue
'''

In [ ]:
from third_party.Neuralangelo.convert_data_to_json import data_to_json

args = {
  "data_dir": str(sfm_dir.absolute()),
  "scene_type": "outdoor",
  "image_dir": str(images.absolute()),
}

data_to_json(args)

if not sfm_images.exists():
  # copy files from input images to self contained project path
  # os.mkdir(new_input_path)
  print("copied input images to self contained SfM reconstruction folder")
  shutil.copytree(images, sfm_images)

## Visualization
We visualize some of the registered images, and color their keypoint by visibility, track length, or triangulated depth.

In [ ]:
#visualization.visualize_sfm_2d(model, images, color_by="visibility", n=5)

In [ ]:
#visualization.visualize_sfm_2d(model, images, color_by="track_length", n=5)

In [ ]:
#visualization.visualize_sfm_2d(model, images, color_by="depth", n=5)